In [1]:
% Purchaseable products
% product(id, name, category, unit_price).
product(1, "Foobar", metasyntactic, 0.99) :- true.
% TODO: add more
?- findall((X,Y,Z,T), product(X,Y,Z,T), Products).

% Asserting clauses for user:product/4


Products = [(1,Foobar,metasyntactic,0.99)]

In [2]:
% Clients
% client(name, phone_number).
client("Alice", 111111) :- true.
client("Bob", 222222).
client("Charlie", 333333).
?- findall((X,Y), client(X,Y), Clients).

% Asserting clauses for user:client/2


Clients = [(Alice,111111),(Bob,222222),(Charlie,333333)]

In [3]:
% Purchase transactions
% purchase(purchase_id, client_phone, product_id, quantity, day_number).
purchase(1, 111111, 1, 5, 1) :- true.
purchase(2, 222222, 1, 2, 1).
purchase(3, 222222, 1, 2, 2).
purchase(4, 333333, 1, 20, 3).
purchase(5, 111111, 1, 5, 4).
purchase(6, 222222, 1, 6000, 4).

?- findall((I,X,Y,Z,T), purchase(I,X,Y,Z,T), Purchases).

% Asserting clauses for user:purchase/5


Purchases = [(1,111111,1,5,1),(2,222222,1,2,1),(3,222222,1,2,2),(4,333333,1,20,3),(5,111111,1,5,4),(6,222222,1,6000,4)]

In [4]:
% Client tiers, based on how much the client has spent
% tier(tier_name, threshold)
tier(none, 0) :- true.
tier(bronze, 1000).
tier(silver, 5000).
tier(gold, 10000).
tier(platinum, 25000).

% Asserting clauses for user:tier/2


In [5]:
% A repeat client is defined as a customer who has multiple purchases made on different days.
repeat_client(PID) :-
    purchase(_,PID, _, _, D1),
    purchase(_,PID, _, _, D2),
    not(D1 = D2).
%    purchase(pid, _f3, _f4, d2).
?- findall(X, repeat_client(X), _RC), list_to_set(_RC, RepeatClients).

% Asserting clauses for user:repeat_client/1


RepeatClients = [111111,222222]

In [6]:
% (internal) The price of a particular purchase is the number of items involved in that purchase,
% multiplied by the cost of an item.
purchase_price(PurchaseId, PurchasePrice) :-
    purchase(PurchaseId, _, ProductId, ProductCount, _),
    product(ProductId, _, _, UnitPrice),
    PurchasePrice is ProductCount*UnitPrice.
?- purchase_price(1, X).

% Asserting clauses for user:purchase_price/2


X = 4.95

In [7]:
% (internal) The list of all purchases on a particular day, along with their prices.
day_purchase_prices(Day, Plist) :- 
    findall(PurchaseId, purchase(PurchaseId, _, _, Count, Day), DayPurchases),  % gather list of all purchases for the day
    findall((PurchaseId, PurchasePrice), % then associate purchaseids with a purchaseprice
        (
            member(PurchaseId, DayPurchases), % for the purchaseids in the purchases of the day only
            purchase_price(PurchaseId, PurchasePrice)
        ),
        Plist).

?- day_purchase_prices(1, P).

% Asserting clauses for user:day_purchase_prices/2


P = [(1,4.95),(2,1.98)]

In [8]:
% The cashflow of a day is defined as the total sum of the purchases made on that day.
cashflow(Day, Amount) :-
    day_purchase_prices(Day, Plist), % First gather the list of purchases with their costs
    findall(Cost, (
        member((_, Cost), Plist),  % then extract the second item from each tuple
        true
    ),AmountList),  % and gather those into a list
    sum_list(AmountList, Amount). % which is then folded

?- cashflow(1, X).

% Asserting clauses for user:cashflow/2


X = 6.93

In [9]:
% (internal) The list of purchases made by a specific client, along with their prices.
client_purchase_prices(Cid, Plist) :- 
    client(_, Cid),  % There must be a client with this id.
    findall(PurchaseId, purchase(PurchaseId, Cid, _, Count, _), ClientPurchases),
    findall((PurchaseId, PurchasePrice),
        (
            member(PurchaseId, ClientPurchases), % for the purchaseids in the purchases of the day only
            purchase_price(PurchaseId, PurchasePrice)
        ),
        Plist).

?- client_purchase_prices(111111, X).

% Asserting clauses for user:client_purchase_prices/2


X = [(1,4.95),(5,4.95)]

In [10]:
% (internal) The total sum paid by the client across all their purchases.
client_spent(Cid, Amount) :-
    client(_, Cid),  % There must be a client with this id.
    client_purchase_prices(Cid, Plist),
    findall(Cost, member((_, Cost), Plist), AmtList),
    sum_list(AmtList, Amount).
?- findall((X,Y), client_spent(X,Y), Spendings).

% Asserting clauses for user:client_spent/2


Spendings = [(111111,9.9),(222222,5943.96),(333333,19.8)]

In [11]:
% The client's tier is defined as the largest tier whose threshold the client's purchase sum has exceeded.
client_tier(Cid, Tier) :-
    client(_, Cid),  % There must be a client with this id.
    client_spent(Cid, Amount),  % Check how much the client spent
    findall(PassedTierAmt, 
    (
        tier(_, PassedTierAmt),  % Get all tier amounts
        PassedTierAmt =< Amount  % that are smaller than that
    ), PassedTierAmts),
    max_list(PassedTierAmts, MaxTierAmt),  % then find the largest one
    tier(Tier, MaxTierAmt).  % and return the tier corresponding to it
    
?- findall((X,Y), client_tier(X,Y), CTiers).

% Asserting clauses for user:client_tier/2


CTiers = [(111111,none),(222222,silver),(333333,none)]